In [110]:
# In[8]:
import math
import numpy as np
import cupy as cp
pool = cp.cuda.MemoryPool(cp.cuda.malloc_managed)
cp.cuda.set_allocator(pool.malloc)
import sys
import os
from scipy.stats import norm
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.path.dirname('__file__'), '..'))

from modulation.modulation import QAMModem

In [31]:
def add_AWGN_GPU(constellation,No):
  # AWGN雑音の生成
  noise = cp.random.normal(0, math.sqrt(No / 2), (len(constellation)))           + 1j * cp.random.normal(0, math.sqrt(No / 2), (len(constellation)))

  # AWGN通信路 = 送信シンボル間干渉が生じないような通信路で送信
  RX_constellation = constellation + noise

  # 以下のprint関数の出力を表示すると、Noとほぼ一致するはず
  #print(cp.dot(noise[0, :], cp.conj(noise[0, :]))/bit_num)

  return RX_constellation

In [99]:
#print(symbol)

#print(mat_symbol.shape)
res=cp.take_along_axis(mat_symbol,info[:,None],axis=1)
const=cp.take_along_axis(mat_symbol,info[:,None],axis=1)
print(const.shape)

const=const[:,0]
print(const.shape)


(100000, 1)
(100000,)


In [100]:
EsNodB=1
EsNo = 10 ** (EsNodB / 10)
No=1/EsNo
count_num=100000
M=4

#make info matrices
info=cp.random.randint(0,M-1,count_num)

#make constellation
modem=QAMModem(M)
tmp=modem.code_book
symbol=cp.zeros(M,dtype=complex)
for i in tmp:
    symbol[modem.bin2de(i)]=tmp[i]

mat_symbol=cp.tile(symbol,(count_num,1))
const=cp.take_along_axis(mat_symbol,info[:,None],axis=1)[:,0]

In [112]:
def make_AMI(EsNodB,M):
  
    EsNo = 10 ** (EsNodB / 10)
    No=1/EsNo
    count_num=100000000

    #make info matrices
    info=cp.random.randint(0,M-1,count_num)

    #make constellation
    modem=QAMModem(M)
    tmp=modem.code_book
    symbol=cp.zeros(M,dtype=complex)
    for i in tmp:
        symbol[modem.bin2de(i)]=tmp[i]
    
    mat_symbol=cp.tile(symbol,(count_num,1))
    const=cp.take_along_axis(mat_symbol,info[:,None],axis=1)[:,0]

    #if cp.any(symbol==const)!=True:
        #print("error")
        #print(symbol)
        #print(const)

    RX_const=add_AWGN_GPU(const,No)
        
    num=cp.sum(cp.exp(-1*cp.abs(np.tile(RX_const,(len(symbol),1))-symbol.reshape(-1,1))**2/No),axis=0)
    
    den=cp.exp(-1*cp.abs(RX_const-const)**2/No)
    H=cp.sum(cp.log2(num/den))
    H/=count_num
    res=math.log2(M)-H
    return res

In [119]:
modem=QAMModem(16)
tmp=modem.ones
tmp[3]

[(-0.9486832980505138+0.31622776601683794j),
 (-0.9486832980505138-0.31622776601683794j),
 (-0.31622776601683794+0.31622776601683794j),
 (-0.31622776601683794-0.31622776601683794j),
 (0.9486832980505138+0.31622776601683794j),
 (0.9486832980505138-0.31622776601683794j),
 (0.31622776601683794+0.31622776601683794j),
 (0.31622776601683794-0.31622776601683794j)]

In [115]:
def make_BMI(EsNodB,M):
      
    EsNo = 10 ** (EsNodB / 10)
    No=1/EsNo
    count_num=100000000

    #make info matrices
    info=cp.random.randint(0,M-1,count_num)

    #make constellation
    modem=QAMModem(M)
    tmp=modem.code_book
    symbol=cp.zeros(M,dtype=complex)
    for i in tmp:
        symbol[modem.bin2de(i)]=tmp[i]
    
    mat_symbol=cp.tile(symbol,(count_num,1))
    const=cp.take_along_axis(mat_symbol,info[:,None],axis=1)[:,0]

    #if cp.any(symbol==const)!=True:
        #print("error")
        #print(symbol)
        #print(const)

    RX_const=add_AWGN_GPU(const,No)
        
    num=cp.sum(cp.exp(-1*cp.abs(np.tile(RX_const,(len(symbol),1))-symbol.reshape(-1,1))**2/No),axis=0)
    
    den=cp.exp(-1*cp.abs(RX_const-const)**2/No)
    H=cp.sum(cp.log2(num/den))
    H/=count_num
    res=math.log2(M)-H
    return res

array(1.12577558)